https://github.com/hidasib/GRU4Rec/blob/master/examples/rsc15/preprocess.py

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import os
os.listdir('./data')

['original', 'preprocessed']

In [3]:
PATH_TO_ORIGINAL_DATA = './data/original/'
PATH_TO_PROCESSED_DATA = './data/preprocessed/'

In [5]:
data = pd.read_csv(PATH_TO_ORIGINAL_DATA + 'yoochoose-clicks.dat', sep=',',
                   header=None, usecols=[0,1,2])
data.columns = ['SessionId', 'TimeStr', 'ItemId']
data = data.astype({'SessionId' : np.int32, 'TimeStr' : str, 'ItemId' : np.int64})

In [9]:
data['Time'] = data.TimeStr.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp())
del(data['TimeStr'])

In [10]:
data.head()

,SessionId,ItemId,Time
0,1,214536502,1.396835e+09
1,1,214536500,1.396836e+09
2,1,214536506,1.396836e+09
3,1,214577561,1.396836e+09
4,2,214662742,1.396847e+09


In [13]:
session_lengths = data.groupby('SessionId').size()
# only session_lengths > 1
data = data[np.in1d(data.SessionId, session_lengths[session_lengths >1].index)] ## np.in1d(x, y) : if x in y -> True

item_supports = data.groupby('ItemId').size()
# only item count >= 5
data = data[np.in1d(data.ItemId, item_supports[item_supports >= 5].index)]

session_lengths = data.groupby('SessionId').size()
# session_lenghts >=2
data = data[np.in1d(data.SessionId, session_lengths[session_lengths >=2 ].index)]

In [15]:
tmax = data.Time.max()
session_max_times = data.groupby('SessionId').Time.max() #max time by SessionId

#train, test time
session_train = session_max_times[session_max_times < tmax - 86400].index
session_test = session_max_times[session_max_times >= tmax - 86400].index

train = data[np.in1d(data.SessionId, session_train)]
test = data[np.in1d(data.SessionId, session_test)]
test = test[np.in1d(test.ItemId, train.ItemId)] #train에도 있는 ItemId만
tslength = test.groupby('SessionId').size()
test = test[np.in1d(test.SessionId, tslength[tslength>=2].index)]

In [18]:
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

In [19]:
print('Full train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train), train.SessionId.nunique(), train.ItemId.nunique()))
train.to_pickle(PATH_TO_PROCESSED_DATA + 'rsc15_train_full.pkl')
print('Test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(test), test.SessionId.nunique(), test.ItemId.nunique()))
test.to_pickle(PATH_TO_PROCESSED_DATA + 'rsc15_test.pkl')

Full train set
	Events: 31637239
	Sessions: 7966257
	Items: 37483
Test set
	Events: 71222
	Sessions: 15324
	Items: 6751
